# Import necessary libraries

In [2]:
cd ..

/net/server2/homes/sep/fantine/projects/microseismic-detection-ml


In [4]:
import os
import numpy as np
import tensorflow as tf
from gcp_io import gcp_io

In [5]:
datapath = '/scr1/fantine/microseismic-detection-ml'

# Consolidate predictions

In [6]:
logits_files = tf.io.gfile.glob(os.path.join(datapath, 'continuous_data/*_logits.npy'))
data_files = [filename.replace('_logits.npy', '.npy') for filename in logits_files]

In [7]:
i = 0
logits = gcp_io.read(logits_files[i])
data = gcp_io.read(data_files[i])

In [8]:
print(logits.shape)
print(data.shape)

(7681620, 1)
(1024, 3615000)


In [22]:
overlap = 0.5 # replace me
input_shape = (512, 128) # replace me 
n1, n2 = data.shape[0], data.shape[1]
w1, w2 = input_shape
d1 = int((1. - overlap) * w1)
d2 = int((1. - overlap) * w2)
# axis 1 is channels, axis 2 is time
dt = 0.0005

In [23]:
predictions = tf.round(tf.nn.sigmoid(logits)).numpy()
predictions = predictions.reshape((len(range(0, n1 - w1 + 1, d1)), -1))

In [26]:
consolidated = np.amax(predictions, axis=0) # collapse the channel axis
indices = np.where(consolidated == 1)[0] # remove the zeros
shifted_predictions = (indices * w2 * (1. - overlap) + w2 // 2) / 500

In [27]:
np.amax(shifted_predictions)

327748.864